In [1]:
import psycopg2


conn_string= "host='localhost' port=54320 dbname='my_database' user='root' password='postgres'" 

with psycopg2.connect(conn_string) as conn, conn.cursor() as cursor:
    query = 'select * from LINEITEM limit 1' 
    cursor.execute(query) 
    result = cursor.fetchone() 
    print(result)

(1, 155190, 7706, 1, Decimal('17.00'), Decimal('21168.23'), Decimal('0.04'), Decimal('0.02'), 'N', 'O', datetime.date(1996, 3, 13), datetime.date(1996, 2, 12), datetime.date(1996, 3, 22), 'DELIVER IN PERSON        ', 'TRUCK     ', 'egular courts above the')


In [7]:
import psycopg2

conn_string= "host='localhost' port=54320 dbname='my_database' user='root' password='postgres'" 

with psycopg2.connect(conn_string) as conn, conn.cursor() as cursor:
    row = (-999, -999, -999, -999, 0, 0, 0, 0, 0, 0, '2016-01-05', '2016-01-05', '2016-01-05', 'SHIPINSTRUCT', 'SHIPMODE', 'COMMENT') # кортеж для вставки
    insert = """
    insert into LINEITEM (
           L_ORDERKEY   
         , L_PARTKEY        
         , L_SUPPKEY   
         , L_LINENUMBER  
         , L_QUANTITY     
         , L_EXTENDEDPRICE    
         , L_DISCOUNT
         , L_TAX
         ,L_RETURNFLAG 
         ,L_LINESTATUS
         ,L_SHIPDATE
         ,L_COMMITDATE
         ,L_RECEIPTDATE
         ,L_SHIPINSTRUCT
         ,L_SHIPMODE
         ,L_COMMENT 
         
    )
    values
    ({values})
    """.format(
    # биндим параметры в строку, чтобы избежать sql-иньекций (best practice)
    values=cursor.mogrify("%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s", row).decode('utf-8')
    )
    print('sql запрос:', insert)
    cursor.execute(insert)
    conn.commit() # комитим транзакцию
    cursor.execute('select * from LINEITEM where L_ORDERKEY < 0')
    result = cursor.fetchall()
    print('результат вставки:', result)

sql запрос: 
    insert into LINEITEM (
           L_ORDERKEY   
         , L_PARTKEY        
         , L_SUPPKEY   
         , L_LINENUMBER  
         , L_QUANTITY     
         , L_EXTENDEDPRICE    
         , L_DISCOUNT
         , L_TAX
         ,L_RETURNFLAG 
         ,L_LINESTATUS
         ,L_SHIPDATE
         ,L_COMMITDATE
         ,L_RECEIPTDATE
         ,L_SHIPINSTRUCT
         ,L_SHIPMODE
         ,L_COMMENT 
         
    )
    values
    ( -999,  -999,  -999,  -999, 0, 0, 0, 0, 0, 0, '2016-01-05', '2016-01-05', '2016-01-05', 'SHIPINSTRUCT', 'SHIPMODE', 'COMMENT')
    
результат вставки: [(-999, -999, -999, -999, Decimal('0.00'), Decimal('0.00'), Decimal('0.00'), Decimal('0.00'), '1', '1', datetime.date(2016, 1, 5), datetime.date(2016, 1, 5), datetime.date(2016, 1, 5), 'SHIPINSTRUCT             ', 'SHIPMODE  ', 'COMMENT'), (-999, -999, -999, -999, Decimal('0.00'), Decimal('0.00'), Decimal('0.00'), Decimal('0.00'), '0', '0', datetime.date(2016, 1, 5), datetime.date(2016, 1, 5

In [8]:
import psycopg2

conn_string= "host='localhost' port=54320 dbname='my_database' user='root' password='postgres'" 
with psycopg2.connect(conn_string) as conn, conn.cursor() as cursor:
    q = "COPY LINEITEM TO STDOUT WITH DELIMITER ',' CSV HEADER;"
    with open('lineitem.csv', 'w') as f:
        cursor.copy_expert(q, f)


In [10]:
import pandas as pd

pd.read_csv('lineitem.csv')

,l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment
0,1,155190,7706,1,17.0,21168.23,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER IN PERSON,TRUCK,egular courts above the
1,1,67310,7311,2,36.0,45983.16,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE BACK RETURN,MAIL,ly final dependencies: slyly bold
2,1,63700,3701,3,8.0,13309.60,0.10,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE BACK RETURN,REG AIR,"riously. regular, express dep"
3,1,2132,4633,4,28.0,28955.64,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR,lites. fluffily even de
4,1,24027,1534,5,24.0,22824.48,0.10,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB,pending foxes. slyly re
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12002428,6000000,32255,2256,1,5.0,5936.25,0.04,0.03,N,O,1996-11-02,1996-11-19,1996-12-01,TAKE BACK RETURN,MAIL,carefully
12002429,6000000,96127,6128,2,28.0,31447.36,0.01,0.02,N,O,1996-09-22,1996-10-01,1996-10-21,NONE,AIR,ooze furiously about the pe
12002430,-999,-999,-999,-999,0.0,0.00,0.00,0.00,1,1,2016-01-05,2016-01-05,2016-01-05,SHIPINSTRUCT,SHIPMODE,COMMENT
12002431,-999,-999,-999,-999,0.0,0.00,0.00,0.00,0,0,2016-01-05,2016-01-05,2016-01-05,SHIPINSTRUCT,SHIPMODE,COMMENT


In [11]:
import psycopg2

conn_string= "host='localhost' port=54320 dbname='my_database' user='root' password='postgres'" 
with psycopg2.connect(conn_string) as conn, conn.cursor() as cursor:
    q = "COPY lineitem from STDIN WITH DELIMITER ',' CSV HEADER;"
    with open('lineitem.csv', 'r') as f:
        cursor.copy_expert(q, f)

In [12]:
import psycopg2

conn_string= "host='localhost' port=54320 dbname='my_database' user='root' password='postgres'" 
with psycopg2.connect(conn_string) as conn, conn.cursor() as cursor:
    cursor.execute('select count(*) from lineitem')
    print(cursor.fetchall())

[(24004866,)]
